# Training and testing target-specific machine-learning models for structure-based virtual screening

This Jupyter notebook helps users train and test target-specific classification-based machine-learning models for structure-based virtual screening, with or without hyperparameter tuning, using PLEC or GRID features. For regression-based models, it is necessary to import relevant libraries and packages and adjust the code and search spaces accordingly.

Additional information can be found in our Nature Protocols paper: Tran-Nguyen, V. K., Junaid, M., Simeon, S. & Ballester, P. J. A practical guide to machine-learning scoring for structure-based virtual screening. Nat. Protoc. (2023)

We recommend users to set up the protocol-env environment before running the code in this Jupyter notebook. This can be done using the protocol-env.yml file in our MLSF-protocol github repository: https://github.com/vktrannguyen/MLSF-protocol.

## 1. Import all necessary Python packages

The following libraries/packages/toolkits need to be installed beforehand: jupyter notebook, pandas, oddt, sklearn, xgboost, rdkit, deepchem, joblib, tqdm, glob, tensorflow. 

In [ ]:
import os
import numpy as np
import pandas as pd
import oddt
import oddt.pandas as opd
from oddt.pandas import ChemDataFrame
from oddt.fingerprints import PLEC
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.utils import parallel_backend
from xgboost.sklearn import XGBClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc
from deepchem.utils import download_url, load_from_disk
from deepchem.utils.vina_utils import prepare_inputs
from deepchem.models import AtomicConvModel
from deepchem.feat import RdkitGridFeaturizer
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import tempfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop
import hyperopt
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval

## 2. Load data tables for the training set and the test set 

Each data table (in the csv file format) needs to consist of at least three columns: 
- "mol_name": provide the identifier (IDs) of each molecule
- "activity": state whether each molecule is an "Active" or an "Inactive"
- "potency": provide the potency value (if available) of each molecule: pIC50, pEC50, pKd, or pKi (all active concentrations in mol/l before logarithmic conversion)

Examples of these csv data files are provided in our MLSF-protocol github repository: https://github.com/vktrannguyen/MLSF-protocol. Please refer to our Nature Protocols paper cited above for more information.

In [ ]:
#Provide the pathway to the csv training data file:
train_data = pd.read_csv("pathway_to_training-set_csv_data_file")

#Provide the pathway to the csv test data file:
test_data = pd.read_csv("pathway_to_test-set_csv_data_file")

#Call the "activity" labels of all training and test molecules:
Train_Class = train_data['activity']
Test_Class = test_data['activity']

## 3. Train and test target-specific machine-learning models

Examples given in this notebook involve:

- Two featurization schemes: protein-ligand extended connectivity (PLEC) fingerprints, GRID features
- Five machine-learning algorithms: random forest (RF), extreme gradient boosting (XGB), support vector machine (SVM), artificial neural network (ANN), and deep neural network (DNN)

### 3.1. Featurization scheme 1: PLEC fingerprints

#### 3.1.1. Import training and test molecules along with target structure(s)

- The structures of all docked molecules are in the mol2 (multi-mol2) file format.
- The target structure(s) is (are) in the mol2 file format. The target structure for training may be different from that for testing.

In [ ]:
#Provide the pathway to the training molecules (mol2 or multi-mol2):
train_mol2 = opd.read_mol2("pathway_to_training-set_mol2_file_after_docking")
train_mol2.columns = ['mol', 'mol_name']
train = train_mol2.merge(train_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')

#Provide the pathway to the test molecules (mol2 or multi-mol2):
test_mol2 = opd.read_mol2("pathway_to_test-set_mol2_file_after_docking")
test_mol2.columns = ['mol', 'mol_name']
test = test_mol2.merge(test_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')

#Extract the structures of all training and test molecules:
train_mols = train['mol']
test_mols = test['mol']

#Provide the pathway(s) to the training and set target/receptor structure(s):
receptor_train = next(oddt.toolkit.readfile('mol2', 'pathway_to_receptor_mol2_structure_for_training'))
receptor_test = next(oddt.toolkit.readfile('mol2', 'pathway_to_receptor_mol2_structure_for_testing'))

#### 3.1.2. Extract PLEC fingerprints from input data

In [ ]:
#Users may keep only one of the following two functions if the training target/receptor structure is the same as the test target/receptor structure
#Users may change the parameters (size, depth_protein, depth_ligand, distance_cutoff) if they wish
def parallel_plec_train(mol):
    feature = PLEC(mol, protein = receptor_train, size = 4092, 
                  depth_protein = 5, depth_ligand = 1,
                  distance_cutoff = 4.5, sparse = False)
    return feature

def parallel_plec_test(mol):
    feature = PLEC(mol, protein = receptor_test, size = 4092, 
                  depth_protein = 5, depth_ligand = 1,
                  distance_cutoff = 4.5, sparse = False)
    return feature

#Users may choose another number of cores (num_cores) that corresponds to their computing resources
num_cores = 20
train_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(parallel_plec_train)(mol) for mol in tqdm(train_mols))
test_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(parallel_plec_test)(mol) for mol in tqdm(test_mols))

#### 3.1.3. Train and test the RF algorithm
3.1.3.1. Without hyperparameter tuning

In [ ]:
#Train the RF model on the training molecules:
rf_plec = RandomForestClassifier(n_estimators = 400, max_features = 'sqrt', n_jobs = 30)
rf_plec.fit(train_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(test_features)
prediction_test_rf_plec_prob = rf_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_plec_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})
plec_result_rf.to_csv("where_you_want_to_store_the_csv_output_file")

3.1.3.2. With hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"n_estimators": hp.uniform("n_estimators", 100, 10000),
         "max_depth": hp.choice("max_depth", [1, 2, 3, 4, 5, None]),
         "criterion": hp.choice("criterion", ['gini', 'entropy'])}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_randomforest(space):
    model = RandomForestClassifier(n_estimators = int(space['n_estimators']),
                                   max_depth = space['max_depth'],
                                   criterion = space['criterion'], n_jobs = 40)
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_rf_classification = fmin(fn = hyperparameter_tuning_randomforest, space = space, algo = tpe.suggest,
                              max_evals = 10, trials = trials)
best_params = space_eval(space, best_rf_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the RF model on the training molecules, using optimal parameters:
rf_plec = RandomForestClassifier(n_estimators = int(best_params['n_estimators']), 
                                 max_depth = best_params['max_depth'], 
                                 criterion = best_params['criterion'],
                                 max_features = 'sqrt', n_jobs = 30)
rf_plec.fit(train_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(test_features)
prediction_test_rf_plec_prob = rf_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_plec_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})
plec_result_rf.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.1.4. Train and test the XGB algorithm
3.1.4.1. Without hyperparameter tuning

In [ ]:
#Train the XGB model on the training molecules:
xgb_plec = XGBClassifier(n_estimators = 1000, n_jobs = 40)
xgb_plec.fit(np.array(train_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(test_features))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(test_features))

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})
plec_result_xgb.to_csv("where_you_want_to_store_the_csv_output_file")

3.1.4.2. With hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"max_depth": hp.uniform("max_depth", 3, 18),
         "gamma": hp.uniform("gamma", 1, 9),
         "reg_alpha": hp.uniform("reg_alpha", 40, 180),
         "reg_lambda": hp.uniform("reg_lambda", 0, 1), 
         "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
         "min_child_weight": hp.uniform("min_child_weight", 0, 10),
         "n_estimators": hp.uniform("n_estimators", 1000, 5000)} 

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_XGB(space):
    model = XGBClassifier(objective = "binary:logistic", 
                          max_depth = int(space['max_depth']),
                          gamma = int(space["gamma"]),
                          reg_alpha = int(space["reg_alpha"]),
                          reg_lambda = space['reg_lambda'],
                          colsample_bytree = space["colsample_bytree"],
                          min_child_weight = int(space["min_child_weight"]),
                          n_estimators = int(space['n_estimators']))
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_xgb_classification = fmin(fn = hyperparameter_tuning_XGB, space = space, algo = tpe.suggest, 
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_xgb_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the XGB model on the training molecules, using optimal parameters:
xgb_plec = XGBClassifier(objective = "binary:logistic",
                         max_depth = int(best_params['max_depth']),
                         gamma = int(best_params['gamma']),
                         reg_alpha = int(best_params['reg_alpha']),
                         reg_lambda = best_params['reg_lambda'],
                         colsample_bytree = best_params['colsample_bytree'],
                         min_child_weight = int(best_params['min_child_weight']),
                         n_estimators = int(best_params['n_estimators']),
                         n_jobs = 40, random_state = 0)
xgb_plec.fit(np.array(train_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(test_features))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(test_features))

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})
plec_result_xgb.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.1.5. Train and test the SVM algorithm
3.1.5.1. Without hyperparameter tuning

In [ ]:
#Train the SVM model on the training molecules:
svm_plec = SVC(degree = 3, kernel = "rbf", probability = True)
svm_plec.fit(train_features, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(test_features)
prediction_test_svm_plec_prob = svm_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Active_Prob": prediction_test_svm_plec_prob[:, 0],
                                 "Inactive_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})
plec_result_svm.to_csv("where_you_want_to_store_the_csv_output_file")

3.1.5.2. With hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"C": hp.uniform("C", 0, 20),
         "gamma": hp.choice("gamma", ['scale', 'auto']),
         "kernel": hp.choice("kernel", ['rbf', 'poly', 'sigmoid'])}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_SVM(space):
    with parallel_backend(backend = "multiprocessing", n_jobs = 40):
        model = SVC(C = space['C'],
                    gamma = space['gamma'],
                    kernel = space['kernel'])
        model.fit(np.array(train_features), Train_Class)
        predicted_train = model.predict(np.array(train_features))
        mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
        
#Search for optimal parameters:
trials = Trials()
best_svm_classification = fmin(fn = hyperparameter_tuning_SVM, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_svm_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the SVM model on the training molecules, using optimal parameters:
svm_plec = SVC(C = best_params['C'], gamma = best_params['gamma'], kernel = best_params['kernel'], 
               probability = True, random_state = 0)
svm_plec.fit(train_features, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(test_features)
prediction_test_svm_plec_prob = svm_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Active_Prob": prediction_test_svm_plec_prob[:, 0],
                                 "Inactive_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})
plec_result_svm.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.1.6. Train and test the ANN algorithm
3.1.6.1. Without hyperparameter tuning

In [ ]:
#Train the ANN model on the training molecules:
ann_plec = MLPClassifier(max_iter = 9000)
ann_plec.fit(train_features, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(test_features)
prediction_test_ann_plec_prob = ann_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})
plec_result_ann.to_csv("where_you_want_to_store_the_csv_output_file")

3.1.6.2. With hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"hidden_layer_sizes": hp.uniform("hidden_layer_sizes", 8, 140),
         "activation": hp.choice("activation", ['relu', 'tanh']),
         "max_iter": hp.uniform("max_iter", 1000, 10000)}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_ANN(space):
    model = MLPClassifier(hidden_layer_sizes = int(space['hidden_layer_sizes']),
                          activation = space['activation'],
                          max_iter = int(space['max_iter']))
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_ann_classification = fmin(fn = hyperparameter_tuning_ANN, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_ann_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the ANN model on the training molecules, using optimal parameters:
ann_plec = MLPClassifier(hidden_layer_sizes = int(best_params['hidden_layer_sizes']), 
                         activation = best_params['activation'], 
                         max_iter = int(best_params['max_iter']), 
                         random_state = 0)
ann_plec.fit(train_features, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(test_features)
prediction_test_ann_plec_prob = ann_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})
plec_result_ann.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.1.7. Train and test the DNN algorithm
3.1.7.1. Without hyperparameter tuning

In [ ]:
#Train the DNN model on the training molecules:
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")
tf.random.set_seed(0)
dnn_plec = keras.Sequential([
    layers.Dense(8192, kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(0),
    layers.Dense(4069, activation = "relu", kernel_regularizer = regularizers.l2(0)),
    layers.BatchNormalization(),
    layers.Dense(2048, activation = "relu"),
    layers.Dropout(0),
    layers.Dense(1, activation = "sigmoid")
])
dnn_plec.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_plec.fit(np.array(train_features), y_train, epochs = 100, batch_size = 500, verbose = False)

#Test the DNN model on the test molecules:
prediction_test_dnn_plec_prob = dnn_plec.predict(np.array(test_features))
prediction_test_dnn_plec_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_plec_prob]

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_dnn = pd.DataFrame({"Active_Prob": prediction_test_dnn_plec_prob[:, 0],
                                "Predicted_Class": prediction_test_dnn_plec_class,
                                "Real_Class": Test_Class})
plec_result_dnn.to_csv("where_you_want_to_store_the_csv_output_file")

3.1.7.2. With hyperparameter tuning



In [ ]:
#Define the search space for optimal parameters:
space = {'choice': hp.choice('num_layers',
                            [ {'layers': 'two'},
                              {'layers': 'three',
                               'units4': hp.uniform('units4', 64, 8192), 
                               'dropout3': hp.uniform('dropout3', 0, 1)}
                            ]),
        'units1': hp.uniform('units1', 64, 8192),
        'units2': hp.uniform('units2', 64, 8192),
        'units3': hp.uniform('units3', 64, 8192),
        'dropout1': hp.uniform('dropout1', 0, 1),
        'dropout2': hp.uniform('dropout2', 0, 1),
        'batch_size': hp.uniform('batch_size', 128, 500),
        'nb_epochs': 100,
        'optimizer': hp.choice('optimizer', ['Adadelta','Adam','rmsprop']),
        'activation': 'relu'
        }
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_DNN(space):
    model = keras.Sequential([
        layers.Dense(abs(int(space['units1'])), kernel_regularizer = regularizers.l2(0), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dropout(space['dropout1']),
        layers.Dense(abs(int(space['units2'])), kernel_regularizer = regularizers.l2(0), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dense(abs(int(space['units3'])), kernel_regularizer = regularizers.l2(0), activation = "relu"),
        layers.Dropout(space['dropout2']),
        layers.Dense(1, activation = "sigmoid")
    ])
    model.compile(optimizer = 'rmsprop', loss = "binary_crossentropy", metrics = ['accuracy'])
    model.fit(np.array(train_features), y_train, epochs = 100, batch_size = int(space['batch_size']), verbose = False)
    predict_proba = model.predict(np.array(train_features))[:, 0]
    predicted_train = ['Active' if num >= 0.5 else "Inactive" for num in predict_proba]
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}

#Search for optimal parameters:
trials = Trials()
best_dnn_classification = fmin(hyperparameter_tuning_DNN, space, algo = tpe.suggest, 
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_dnn_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the DNN model on the training molecules, using optimal parameters:
tf.random.set_seed(0)
dnn_plec = keras.Sequential([
    layers.Dense(units = int(best_params['units1']), kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(best_params['dropout1']),
    layers.Dense(units = int(best_params['units2']), kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dense(units = int(best_params['units3']), kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.Dropout(best_params['dropout2']),
    layers.Dense(1, activation = "sigmoid")    
])
dnn_plec.compile(optimizer = best_params['optimizer'], loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_plec.fit(np.array(train_features), y_train, epochs = 100, 
             batch_size = int(best_params['batch_size']), verbose = False)

#Test the DNN model on the test molecules:
prediction_test_dnn_plec_prob = dnn_plec.predict(np.array(test_features))
prediction_test_dnn_plec_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_plec_prob]

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_dnn = pd.DataFrame({"Active_Prob": prediction_test_dnn_plec_prob[:, 0],
                                "Predicted_Class": prediction_test_dnn_plec_class,
                                "Real_Class": Test_Class})
plec_result_dnn.to_csv("where_you_want_to_store_the_csv_output_file")

### 3.2. Featurization scheme 2: Grid features
#### 3.2.1. Import training and test molecules along with target structure(s)
- The structures of all docked molecules are in the sdf (multi-sdf) file format.
- The target structure(s) is (are) in the pdb file format. The target structure for training may be different from that for testing.

In [ ]:
#Provide the pathway to the training molecules (sdf or multi-sdf):
train_sdf = opd.read_sdf("pathway_to_the_training-set_sdf_after_docking")
train = train_sdf.merge(train_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')

#Split the multi-sdf training data into individual sdf structures:
for i in range(train_sdf.shape[0]):
    each_sdf = train_sdf.iloc[[i]]
    each_name = list(each_sdf['mol_name'])[0]
    out_path = '/pathway_to_the_directory_to_store_individual_training-set_sdfs/' + str(each_name) + '.sdf'
    ChemDataFrame.to_sdf(each_sdf, out_path)
training_set = glob.glob("/pathway_to_the_directory_to_store_individual_training-set_sdfs/*")

#Provide the pathway to the test molecules (sdf or multi-sdf):
test_sdf = opd.read_sdf("pathway_to_the_test-set_sdf_after_docking")
test = test_sdf.merge(test_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')

#Split the multi-sdf test data into individual sdf structures:
for i in range(test_sdf.shape[0]):
    each_sdf = test_sdf.iloc[[i]]
    each_name = list(each_sdf['mol_name'])[0]
    out_path = '/pathway_to_the_directory_to_store_individual_test-set_sdfs/' + str(each_name) + '.sdf'
    ChemDataFrame.to_sdf(each_sdf, out_path)
test_set = glob.glob("/pathway_to_the_directory_to_store_individual_test-set_sdfs/*")

#Extract the structures of all training and test molecules:
train_mols = train['mol']
test_mols = test['mol']

#Provide the pathway(s) to the training and set target/receptor structure(s):
protein_train = "pathway_to_receptor_pdb_file_for_training"
protein_test = "pathway_to_receptor_pdb_file_for_testing"

In [ ]:
#Users may keep only one of the following two functions if the training target/receptor structure is the same as the test target/receptor structure
#Users may change the parameters (voxel_width, feature_types, ecfp_power, splif_power) if they wish
featurizer = RdkitGridFeaturizer(voxel_width = 16.0, feature_types = ["ecfp", "splif", "hbond", "salt_bridge"], ecfp_power = 9, splif_power = 9, flatten = True, verbose = False)
def extract_grid_feature_train(ligand_file):
    try:
        feature = featurizer._featurize((ligand_file, protein_train))
        return feature
    except:
        pass
def extract_grid_feature_test(ligand_file):
    try:
        feature = featurizer._featurize((ligand_file, protein_test))
        return feature
    except:
        pass

#Users may choose another number of cores (num_cores) that corresponds to their computing resources
num_cores = 20
train_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(extract_grid_feature_train)(ligand_file) for ligand_file in tqdm(training_set))
test_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(extract_grid_feature_test)(ligand_file) for ligand_file in tqdm(test_set))

#Export grid features as npy files:
np.save('where_you_want_to_store_the_npy_file_containing_GRID_features_of_training-set', train_features)
np.save('where_you_want_to_store_the_npy_file_containing_GRID_features_of_test-set', test_features)

#Load grid features for the next stages:
train_grid_features = np.load("where_you_want_to_store_the_npy_file_containing_GRID_features_of_training-set")
test_grid_features = np.load("where_you_want_to_store_the_npy_file_containing_GRID_features_of_test-set")

The following part is necessary for the hyperparameter tuning process only

In [ ]:
#Normalize grid features for hyperparameter tuning:
train_rdkit_grid_features = pd.DataFrame(train_grid_features)
test_rdkit_grid_features = pd.DataFrame(test_grid_features)

def normalize(feature):
    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(feature)
    fp_normalize = pd.DataFrame(np_scaled)
    return fp_normalize

train_rdkit_grid_features = normalize(train_rdkit_grid_features)
test_rdkit_grid_features = normalize(test_rdkit_grid_features)

#### 3.2.3. Train and test the RF algorithm
3.2.3.1. Without hyperparameter tuning

In [ ]:
#Train the RF model on the training molecules:
rf_grid = RandomForestClassifier(n_estimators = 500, n_jobs = 30, random_state = 0)
rf_grid.fit(train_grid_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_grid_class = rf_grid.predict(test_grid_features)
prediction_test_rf_grid_prob = rf_grid.predict_proba(test_grid_features)

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_grid_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_grid_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_grid_class,
                               "Real_Class": Test_Class})
grid_result_rf.to_csv("where_you_want_to_store_the_csv_output_file")

3.2.3.2. With hyperparameter tuning



In [ ]:
#Define the search space for optimal parameters:
space = {"n_estimators": hp.uniform("n_estimators", 100, 10000),
         "max_depth": hp.choice("max_depth", [1, 2, 3, 4, 5, None]),
         "criterion": hp.choice("criterion", ['gini', 'entropy'])}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_randomforest(space):
    model = RandomForestClassifier(n_estimators = int(space['n_estimators']),
                                   max_depth = space['max_depth'],
                                   criterion = space['criterion'], n_jobs = 40)
    model.fit(np.array(train_rdkit_grid_features), Train_Class)
    predicted_train = model.predict(train_rdkit_grid_features)
    mcc = matthews_corrcoef(predicted_train, Train_Class)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}

#Search for optimal parameters:
trials = Trials()
best_rf_classification = fmin(hyperparameter_tuning_randomforest, space, algo = tpe.suggest, 
                              max_evals = 10, trials = trials)
best_params = space_eval(space, best_rf_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the RF model on the training molecules, using optimal parameters:
rf_grid = RandomForestClassifier(n_estimators = int(best_params['n_estimators']), 
                                 max_depth = best_params['max_depth'], 
                                 criterion = best_params['criterion'],
                                 max_features = 'sqrt', n_jobs = 30, random_state = 0)
rf_grid.fit(train_rdkit_grid_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_grid_class = rf_grid.predict(test_rdkit_grid_features)
prediction_test_rf_grid_prob = rf_grid.predict_proba(test_rdkit_grid_features)

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_grid_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_grid_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_grid_class,
                               "Real_Class": Test_Class})
grid_result_rf.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.2.4. Train and test the XGB algorithm
3.2.4.1. Without hyperparameter tuning

In [ ]:
#Train the XGB model on the training molecules:
xgb_grid = XGBClassifier(n_estimators = 745, max_depth = 7, learning_rate = 0.05, n_jobs = 40, random_state = 0)
xgb_grid.fit(np.array(train_grid_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_grid_class = xgb_grid.predict(np.array(test_grid_features))
prediction_test_xgb_grid_prob = xgb_grid.predict_proba(np.array(test_grid_features))

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_grid_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_grid_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_grid_class,
                                "Real_Class": Test_Class})
grid_result_xgb.to_csv("where_you_want_to_store_the_csv_output_file")

3.2.4.2. With hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"max_depth": hp.uniform("max_depth", 3, 18),
         "gamma": hp.uniform("gamma", 1, 9),
         "reg_alpha": hp.uniform("reg_alpha", 40, 180),
         "reg_lambda": hp.uniform("reg_lambda", 0, 1), 
         "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
         "min_child_weight": hp.uniform("min_child_weight", 0, 10),
         "n_estimators": hp.uniform("n_estimators", 1000, 5000)} 

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_XGB(space):
    model = XGBClassifier(objective = "binary:logistic", 
                          max_depth = int(space['max_depth']),
                          gamma = int(space["gamma"]),
                          reg_alpha = int(space["reg_alpha"]),
                          reg_lambda = space['reg_lambda'],
                          colsample_bytree = space["colsample_bytree"],
                          min_child_weight = int(space["min_child_weight"]),
                          n_estimators = int(space['n_estimators']))
    model.fit(np.array(train_rdkit_grid_features), Train_Class)
    predicted_train = model.predict(np.array(train_rdkit_grid_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}

#Search for optimal parameters:
trials = Trials()
best_xgb_classification = fmin(fn = hyperparameter_tuning_XGB, space = space, algo = tpe.suggest, 
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_xgb_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the XGB model on the training molecules, using optimal parameters:
xgb_grid = XGBClassifier(objective = "binary:logistic",
                         max_depth = int(best_params['max_depth']),
                         gamma = int(best_params['gamma']),
                         reg_alpha = int(best_params['reg_alpha']),
                         reg_lambda = best_params['reg_lambda'],
                         colsample_bytree = best_params['colsample_bytree'],
                         min_child_weight = int(best_params['min_child_weight']),
                         n_estimators = int(best_params['n_estimators']),
                         n_jobs = 40, random_state = 0)
xgb_grid.fit(np.array(train_rdkit_grid_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_grid_class = xgb_grid.predict(np.array(test_rdkit_grid_features))
prediction_test_xgb_grid_prob = xgb_grid.predict_proba(np.array(test_rdkit_grid_features))

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_grid_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_grid_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_grid_class,
                                "Real_Class": Test_Class})
grid_result_xgb.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.2.5. Train and test the SVM algorithm
3.2.5.1. Without hyperparameter tuning

In [ ]:
#Train the SVM model on the training molecules:
svm_grid = SVC(degree = 3, kernel = "rbf", probability = True, random_state = 0)
svm_grid.fit(train_grid_features, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_grid_class = svm_grid.predict(test_grid_features)
prediction_test_svm_grid_prob = svm_grid.predict_proba(test_grid_features)

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_svm = pd.DataFrame({"Active_Prob": prediction_test_svm_grid_prob[:, 0],
                                "Inactive_Prob": prediction_test_svm_grid_prob[:, 1],
                                "Predicted_Class": prediction_test_svm_grid_class,
                                "Real_Class": Test_Class})
grid_result_svm.to_csv("where_you_want_to_store_the_csv_output_file")

3.2.5.2. With hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"C": hp.uniform("C", 1, 20),
         "gamma": hp.choice("gamma", ['scale', 'auto']),
         "kernel": hp.choice("kernel", ['rbf', 'poly', 'sigmoid'])}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_SVM(space):
    with parallel_backend(backend = "multiprocessing", n_jobs = 40):
        model = SVC(C = space['C'],
                    gamma = space['gamma'],
                    kernel = space['kernel'], probability = True)
        model.fit(np.array(train_rdkit_grid_features), Train_Class)
        predicted_train = model.predict(np.array(train_rdkit_grid_features))
        mcc = matthews_corrcoef(predicted_train, Train_Class)
    return {'loss': 1-mcc, 'status': STATUS_OK, "model": model}

#Search for optimal parameters:
trials = Trials()
best_svm_classification = fmin(fn = hyperparameter_tuning_SVM, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_svm_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the SVM model on the training molecules, using optimal parameters:
svm_grid = SVC(C = best_params['C'], gamma = best_params['gamma'], kernel = best_params['kernel'], 
               probability = True, random_state = 0)
svm_grid.fit(np.array(train_rdkit_grid_features), Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_grid_class = svm_grid.predict(test_rdkit_grid_features)
prediction_test_svm_grid_prob = svm_grid.predict_proba(test_rdkit_grid_features)

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_svm = pd.DataFrame({"Active_Prob": prediction_test_svm_grid_prob[:, 0],
                                "Inactive_Prob": prediction_test_svm_grid_prob[:, 1],
                                "Predicted_Class": prediction_test_svm_grid_class,
                                "Real_Class": Test_Class})
grid_result_svm.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.2.6. Train and test the ANN algorithm
3.2.6.1. Without hyperparameter tuning

In [ ]:
#Train the ANN model on the training molecules:
ann_grid = MLPClassifier(max_iter = 500, random_state = 0)
ann_grid.fit(train_grid_features, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_grid_class = ann_grid.predict(test_grid_features)
prediction_test_ann_grid_prob = ann_grid.predict_proba(test_grid_features)

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_grid_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_grid_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_grid_class,
                                "Real_Class": Test_Class})
grid_result_ann.to_csv("where_you_want_to_store_the_csv_output_file")

3.2.6.2. With hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"hidden_layer_sizes": hp.uniform("hidden_layer_sizes", 8, 140),
         "activation": hp.choice("activation", ['relu', 'tanh']),
         "max_iter": hp.uniform("max_iter", 1000, 10000)}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_ANN(space):
    model = MLPClassifier(hidden_layer_sizes = int(space['hidden_layer_sizes']),
                          activation = space['activation'],
                          max_iter = int(space['max_iter']))
    model.fit(np.array(train_rdkit_grid_features), Train_Class)
    predicted_train = model.predict(np.array(train_rdkit_grid_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {"loss": 1-mcc, "status": STATUS_OK, 'model': model}

#Search for optimal parameters:
trials = Trials()
best_ann_classiifcation = fmin(fn = hyperparameter_tuning_ANN, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_ann_classiifcation)

#Optimal parameters:
best_params

In [ ]:
#Train the ANN model on the training molecules, using optimal parameters:
ann_grid = MLPClassifier(hidden_layer_sizes = int(best_params['hidden_layer_sizes']), 
                         activation = best_params['activation'], 
                         max_iter = int(best_params['max_iter']), 
                         random_state = 0)
ann_grid.fit(np.array(train_rdkit_grid_features), Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_grid_class = ann_grid.predict(np.array(test_rdkit_grid_features))
prediction_test_ann_grid_prob = ann_grid.predict_proba(np.array(test_rdkit_grid_features))

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_grid_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_grid_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_grid_class,
                                "Real_Class": Test_Class})
grid_result_ann.to_csv("where_you_want_to_store_the_csv_output_file")

#### 3.2.7. Train and test the DNN algorithm
3.2.7.1. Without hyperparameter tuning

In [ ]:
#Train the DNN model on the training molecules:
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")
tf.random.set_seed(0)
dnn_grid = keras.Sequential([
    layers.Dense(8192, kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(0),
    layers.Dense(4069, activation = "relu", kernel_regularizer = regularizers.l2(0)),
    layers.BatchNormalization(),
    layers.Dense(2048, activation = "relu"),
    layers.Dropout(0),
    layers.Dense(1, activation = "sigmoid")
])
dnn_grid.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_grid.fit(train_grid_features, y_train, epochs = 100, batch_size = 500, verbose = False)

#Test the DNN model on the test molecules:
prediction_test_dnn_grid_prob = dnn_grid.predict(test_grid_features)
prediction_test_dnn_grid_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_grid_prob]

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_dnn = pd.DataFrame({"Active_Prob": prediction_test_dnn_grid_prob[:, 0],
                                "Predicted_Class": prediction_test_dnn_grid_class,
                                "Real_Class": Test_Class})
grid_result_dnn.to_csv("where_you_want_to_store_the_csv_output_file")

In [ ]:
#Define the search space for optimal parameters:
space = {'choice': hp.choice('num_layers',
                            [ {'layers': 'two'},
                              {'layers': 'three',
                               'units4': hp.uniform('units4', 64, 8192), 
                               'dropout3': hp.uniform('dropout3', 0, 1)}
                            ]),
        'units1': hp.uniform('units1', 64, 8192),
        'units2': hp.uniform('units2', 64, 8192),
        'units3': hp.uniform('units3', 64, 8192),
        'dropout1': hp.uniform('dropout1', 0, 1),
        'dropout2': hp.uniform('dropout2', 0, 1),
        'batch_size': hp.uniform('batch_size', 128, 500),
        'nb_epochs': 100,
        'optimizer': hp.choice('optimizer', ['Adadelta','Adam','rmsprop']),
        'activation': 'relu'
        }
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_DNN(space):
    model = keras.Sequential([
        layers.Dense(abs(int(space['units1'])), kernel_regularizer = regularizers.l2(0), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dropout(space['dropout1']),
        layers.Dense(abs(int(space['units2'])), kernel_regularizer = regularizers.l2(0), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dense(abs(int(space['units3'])), kernel_regularizer = regularizers.l2(0), activation = "relu"),
        layers.Dropout(space['dropout2']),
        layers.Dense(1, activation = "sigmoid")
    ])
    model.compile(optimizer = 'rmsprop', loss = "binary_crossentropy", metrics = ['accuracy'])
    model.fit(np.array(train_rdkit_grid_features), y_train, epochs = 100, batch_size = int(space['batch_size']), 
              verbose = False)
    prediction_train_prob_dnn_grid = model.predict(np.array(train_rdkit_grid_features))[:, 0]
    prediction_train_class_dnn_grid = np.where(prediction_train_prob_dnn_grid >= 0.5, "Active", "Inactive")
    mcc = matthews_corrcoef(Train_Class, prediction_train_class_dnn_grid)
    return {'loss': 1-mcc, "status": STATUS_OK, 'model': model}

#Search for optimal parameters:
trials = Trials()
best_dnn_classification = fmin(fn = hyperparameter_tuning_DNN, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_dnn_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the DNN model on the training molecules, using optimal parameters:
tf.random.set_seed(0)
dnn_grid = keras.Sequential([
    layers.Dense(units = int(best_params['units1']), kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(best_params['dropout1']),
    layers.Dense(units = int(best_params['units2']), kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dense(units = int(best_params['units3']), kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.Dropout(best_params['dropout2']),
    layers.Dense(1, activation = "sigmoid")    
])
dnn_grid.compile(optimizer = best_params['optimizer'], loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_grid.fit(np.array(train_rdkit_grid_features), y_train, epochs = 100, 
             batch_size = int(best_params['batch_size']), verbose = False)

#Test the DNN model on the test molecules:
prediction_test_dnn_grid_prob = dnn_grid.predict(np.array(test_rdkit_grid_features))
prediction_test_dnn_grid_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_plec_prob]

#Get virtual screening results on the test molecules and export results to a csv file:
grid_result_dnn = pd.DataFrame({"Active_Prob": prediction_test_dnn_grid_prob[:, 0],
                                "Predicted_Class": prediction_test_dnn_grid_class,
                                "Real_Class": Test_Class})
grid_result_dnn.to_csv("where_you_want_to_store_the_csv_output_file")